## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
!pip install transformers
!pip install mlflow

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType, FloatType
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name, current_timestamp, col, count, when, udf, explode, split, when, sum as sql_sum
from pyspark.sql.streaming import StreamingQueryListener
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import numpy as np
from time import sleep
import pandas as pd
import matplotlib.pyplot as plt
import re
from transformers import pipeline
import mlflow

## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
# Read the source file
source_files_df = get_source_listing_df()
print(source_files_df)

file_content = show_s3_file_contents("voc_volume/0.json")
print(file_content.decode('utf-8'))

According to the result, there are 20,000 json files. The file contains the information about the post date of the tweet, user name, text and the sentiment.

## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
# Define the schema
schema = StructType([
    StructField("date", StringType(), True),
    StructField("user", StringType(), True),
    StructField("text", StringType(), True),
    StructField("sentiment", StringType(), True),
    StructField("source_file", StringType(), True),
    StructField("processing_time", TimestampType(), True)
])

# Initialize Spark Session
spark = SparkSession.builder.appName("JsonToBronze").getOrCreate()

In [0]:
# Stop with fewer tweets
input_path = "s3a://voc-75-databricks-data/voc_volume/"

# Initialize Spark Session with Delta Lake
spark = SparkSession.builder \
    .appName("CloudFilesToDelta") \
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# the Read Stream
df = spark.readStream \
    .format("cloudFiles") \
    .schema(schema) \
    .option("cloudFiles.format", "json") \
    .load(input_path) \
    .withColumn("source_file", input_file_name()) \
    .withColumn("processing_time", current_timestamp())


def check_tweet_count_bronze(streaming_query, threshold=100):
    while streaming_query.isActive:
        # Load the Delta table to check the current count
        tweet_count = spark.read.format("delta").load(BRONZE_DELTA).count()
        print(f"Current tweet count: {tweet_count}")
        if tweet_count >= threshold:
            print(f"Threshold reached: {tweet_count} tweets. Stopping the stream.")
            streaming_query.stop()
            break
        time.sleep(10)  # Check every 10 seconds

# the write stream if more than 100 tweets
streaming_query = df.writeStream \
    .format("delta") \
    .option("checkpointLocation", BRONZE_CHECKPOINT) \
    .outputMode("append") \
    .queryName("bronze_stream") \
    .start(BRONZE_DELTA)

# Monitor and potentially stop the stream
check_tweet_count_bronze(streaming_query)

## 4.0 Bronze Data Exploratory Data Analysis
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
# Load the Delta table
bronze_df = spark.read.format("delta").load(BRONZE_DELTA)

# Count the number of tweets
tweet_count = bronze_df.count()

# Check the missing values
missing_values_count = bronze_df.select([count(when(col(c).isNull(), c)).alias(c) for c in bronze_df.columns])

missing_values_count.show()


print(f"Number of tweets captured: {tweet_count}")

print("Number of missing values:", missing_values_count)

In [0]:
# Count the number of tweets by each unique user
tweet_counts = bronze_df.groupBy("user").agg(count("text").alias("tweet_count"))

# Sort the results by tweet_count in descending order
sorted_tweet_counts = tweet_counts.orderBy(col("tweet_count").desc())

# To show the top 20 users by tweet count
sorted_tweet_counts.show()

In [0]:
# Count tweets that contain at least one '@' mention
tweets_with_mentions = bronze_df.filter(col("text").contains("@")).count()

# Count tweets that do not contain any '@' mention
tweets_without_mentions = bronze_df.filter(~col("text").contains("@")).count()

print(f"Number of tweets with at least one mention: {tweets_with_mentions}")
print(f"Number of tweets with no mentions: {tweets_without_mentions}")

In [0]:
# Plot the bar chart
# Retrieve top 20 tweeters
top_tweeters = sorted_tweet_counts.limit(20)

top_tweeters_data = top_tweeters.collect()

# Extract user handles and tweet counts
users = [row['user'] for row in top_tweeters_data]
tweet_counts = [row['tweet_count'] for row in top_tweeters_data]

# Create a bar chart
plt.figure(figsize=(10, 8))
plt.bar(users, tweet_counts)
plt.xlabel('Users')
plt.ylabel('Number of Tweets')
plt.title('Top 20 Tweeters')
plt.xticks(rotation=45, ha='right')
plt.tight_layout() 

# Display the plot
plt.show()

## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
# Silver builder
spark = SparkSession.builder \
    .appName("silver_stream") \
    .getOrCreate()

# Load the Bronze Delta table as a stream
bronze_df = spark.readStream \
    .format("delta") \
    .load(BRONZE_DELTA)

In [0]:
def find_mentions(text):
    return re.findall(r'@\w+', text)

def remove_mentions(text):
    return re.sub(r'@\w+', '', text)

find_mentions_udf = udf(find_mentions, StringType())
remove_mentions_udf = udf(remove_mentions, StringType())

# Apply transformations
silver_df = bronze_df \
    .withColumn("timestamp", col("date").cast(TimestampType())) \
    .withColumn("text", col("text").cast(StringType())) \
    .withColumn("mentions", find_mentions_udf(col("text"))) \
    .withColumn("cleaned_text", remove_mentions_udf(col("text")))

# Split the mentions string into an array of strings
silver_df = silver_df.withColumn("mentions_array", split(col("mentions"), ","))

# Explode mentions_array into separate rows
silver_exploded_df = silver_df.withColumn("mention", explode(col("mentions_array")))

In [0]:
# Silver stream write
silver_query = silver_exploded_df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", SILVER_CHECKPOINT) \
    .start(SILVER_DELTA)
try:
    silver_query.processAllAvailable()
finally:
    # Stop the stream
    silver_query.stop()

## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
silver_df = spark.readStream \
    .format("delta") \
    .load(SILVER_DELTA)


In [0]:
# Load the HF model
sentiment_analysis = pipeline("sentiment-analysis", model=HF_MODEL_NAME)

# UDF to get sentiment score and label
@udf(returnType=FloatType())
def get_sentiment_score(text):
    return sentiment_analysis(text)[0]['score']

@udf(returnType=StringType())
def get_sentiment_label(text):
    return sentiment_analysis(text)[0]['label']

@udf(returnType=IntegerType())
def get_sentiment_id(sentiment):
    return 0 if sentiment == 'negative' else 1

# Apply sentiment analysis on the cleaned_text
gold_df_0 = silver_df \
    .withColumn("predicted_score", get_sentiment_score(col("cleaned_text"))) \
    .withColumn("predicted_sentiment", get_sentiment_label(col("cleaned_text"))) \
    .withColumn("predicted_sentiment_id", get_sentiment_id(col("predicted_sentiment"))) \
    .withColumn("sentiment_id", get_sentiment_id(col("sentiment")))

In [0]:
# Write gold stream
spark = SparkSession.builder \
    .appName("gold_stream") \
    .getOrCreate()

def check_tweet_count_gold(streaming_query, threshold=100):
    while streaming_query.isActive:
        # Load the Delta table to check the current count
        tweet_count = spark.read.format("delta").load(GOLD_DELTA).count()
        print(f"Current tweet count: {tweet_count}")
        if tweet_count >= threshold:
            print(f"Threshold reached: {tweet_count} tweets. Stopping the stream.")
            streaming_query.stop()
            break
        time.sleep(10) 

gold_query = gold_df_0.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", GOLD_CHECKPOINT) \
    .option("mergeSchema", "true") \
    .start(GOLD_DELTA)

check_tweet_count_gold(gold_query)


## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
# Set up the MLflow
mlflow.set_tracking_uri("uri_to_your_mlflow_server") 
mlflow.set_experiment("Twitter_Sentiment_Analysis")

In [0]:
spark = SparkSession.builder \
    .appName("MLMetrics") \
    .getOrCreate()

gold_df = spark.readStream \
        .format("delta") \
        .load("/tmp/labuser104917-2387441/gold.delta")

In [0]:
# Start an MLflow run
mlflow.end_run()
mlflow.start_run()

# model details and Delta Table version
model_name = "Sentiment Analysis Model"
mlflow_version = mlflow.__version__
# delta_table_version = spark.read.format("delta").option("versionAsOf", 0).load("/tmp/labuser104917-3007326/gold.delta").version()

# Log MLflow parameters
mlflow.log_param("model_name", model_name)
mlflow.log_param("mlflow_version", mlflow_version)
# mlflow.log_param("delta_table_version", delta_table_version)

batch_count = 0  # Counter for the number of batches processed

def foreach_batch_function(df, epoch_id):
    global batch_count
    batch_count += 1

    if df.count() > 0:
        # Convert to Pandas DataFrame for sklearn operations
        pandas_df = df.select("predicted_sentiment_id", "sentiment_id").toPandas()

        # Generate and plot the confusion matrix
        cm = confusion_matrix(pandas_df['sentiment_id'], pandas_df['predicted_sentiment_id'])
        plt.figure(figsize=(10, 7))
        sns.heatmap(cm, annot=True, fmt='g')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title('Confusion Matrix')
        image_path = f"/tmp/confusion_matrix_{epoch_id}.png"
        plt.savefig(image_path)
        plt.close()

        # Log the confusion matrix as an MLflow artifact
        mlflow.log_artifact(image_path)
        
        # Convert predicted_sentiment_id column to DoubleType
        df = df.withColumn("predicted_sentiment_id", df["predicted_sentiment_id"].cast("double"))
        
        # Calculate metrics using Spark's evaluator
        evaluator = MulticlassClassificationEvaluator(
            labelCol='sentiment_id', 
            predictionCol='predicted_sentiment_id')
        precision = evaluator.evaluate(df, {evaluator.metricName: "precisionByLabel"})
        recall = evaluator.evaluate(df, {evaluator.metricName: "recallByLabel"})
        f1_score = evaluator.evaluate(df, {evaluator.metricName: "f1"})

        # Log metrics to MLflow
        mlflow.log_metric("precision", precision, step=epoch_id)
        mlflow.log_metric("recall", recall, step=epoch_id)
        mlflow.log_metric("f1_score", f1_score, step=epoch_id)

        # Print metrics to console for debugging
        print(f"Batch {epoch_id}: Precision = {precision}, Recall = {recall}, F1 Score = {f1_score}")

    # Stop the query after processing a batch
    if batch_count >= 0:  
        print(f"Processed {batch_count} batches, stopping the stream.")
        query.stop()

# Define the streaming query with the foreachBatch sink
query = gold_df.writeStream \
    .foreachBatch(foreach_batch_function) \
    .outputMode("update") \
    .start()

query.awaitTermination()

# End the MLflow run when the stream stops
mlflow.end_run()


## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
gold_df = spark.readStream \
        .format("delta") \
        .load("/tmp/labuser104917-2387441/gold.delta")

In [0]:
# Define the aggregation expressions for each sentiment type
sentiment_counts = gold_df.withColumn("positive", (col("sentiment") == "positive").cast("integer")) \
                          .withColumn("neutral", (col("sentiment") == "neutral").cast("integer")) \
                          .withColumn("negative", (col("sentiment") == "negative").cast("integer")) \
                          .groupBy("mention") \
                          .agg(
                              sql_sum("positive").alias("positive_count"),
                              sql_sum("neutral").alias("neutral_count"),
                              sql_sum("negative").alias("negative_count"),
                              (sql_sum("positive") + sql_sum("neutral") + sql_sum("negative")).alias("total_mentions")
                          ) \
                          .orderBy(col("total_mentions").desc())  # Sorting by total_mentions in descending order

# Print the schema to understand what the DataFrame looks like
sentiment_counts.printSchema()

In [0]:
query = sentiment_counts.writeStream \
    .queryName("aggregated_sentiments") \
    .outputMode("complete") \
    .format("memory") \
    .start()

try:
    # Process available data before plotting
    query.processAllAvailable()

    # Function to fetch data and plot for specified sentiment
    def plot_sentiment_mentions(sentiment_type):
        column = f"{sentiment_type}_count"
        result_df = spark.sql(f"""
            SELECT mention, {column}
            FROM aggregated_sentiments
            ORDER BY {column} DESC
            LIMIT 20
        """).toPandas()

        plt.figure(figsize=(10, 8))
        plt.barh(result_df['mention'], result_df[column], color='salmon' if sentiment_type == 'negative' else 'lightblue')
        plt.xlabel(f'Count of {sentiment_type.capitalize()} Mentions')
        plt.ylabel('Mention')
        plt.title(f'Top 20 Mentions by {sentiment_type.capitalize()} Sentiment')
        plt.gca().invert_yaxis()  # Invert the y-axis so the largest is on top
        plt.show()

    # Plotting positive and negative sentiments
    plot_sentiment_mentions('positive')
    plot_sentiment_mentions('negative')

finally:
    # Stop the stream
    query.stop()


## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
spark = SparkSession.builder.appName("StreamMonitoring").getOrCreate()

active_streams = spark.streams.active

# Print details about each stream
for stream in active_streams:
    print(f"Stream Name: {stream.name}")
    print(f"Stream ID: {stream.id}")
    print(f"Stream Run ID: {stream.runId}")
    print(f"Is Stream Active: {stream.isActive}")
    print("-" * 40)

In [0]:
for stream in spark.streams.active:
    stream.stop()

dbutils.fs.rm(BRONZE_CHECKPOINT, recurse=True)
dbutils.fs.rm(SILVER_CHECKPOINT, recurse=True)
dbutils.fs.rm(GOLD_CHECKPOINT, recurse=True)

In [0]:
# Get the notebooks ending time note START_TIME was established in the include file when the notebook started.
END_TIME = time.time()
print(END_TIME)

## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

To build and evaluate the pipeline for this project, I primarily use a partial dataset. Specifically, the dataset comprises 133,118 tweets that are processed through the Spark structured streaming pipeline. The answers to the question are derived from data displayed in the Spark UI, which are based on this partial dataset. Additionally, I speculate on potential issues that may arise when the pipeline is executed on the full dataset.

### Spill
This figure shows the memory usage when running the EDA with Bronze data:
https://drive.google.com/file/d/1xQH-rEKlDBFLFS7NNOM8xbra-K9qwITi/view?usp=sharing

According to the figure, the memory usage is 1.7 MiB out of 21.1 GiB, suggesting that there is sufficient memory available to prevent spilling. However, when processing the full dataset or implementing more complex algorithms, there is a higher likelihood of experiencing memory spill. 

### Skew
According to the figures shown in the Session 8.0, the sentiments distribution is imbalanced in the dataset. Solutions to the potential skews include salting keys or increase the number of partitions.

### Shuffle
When using the "groupBy" command in the pipeline, but it can also be resource intensive and time consuming. For handling the full dataset, consider using narrow transformations, which are typically more efficient.

### Storage
The usage of storage from the spark UI is shown in the figure:
https://drive.google.com/file/d/14grQhoeebqmedRMByji5sZh-mV2Aym1D/view?usp=sharing

It shows that the disk usage is 36.9 MiB out of 463.9 GiB. There is enough storage space with the current partial data. If using the full dataset, we could expand the cluster storage or restrict the cache to the data that will only be used repeatedly.

### Serialization
The issue may arise because serialization can be computationally demanding. With partial data, serialization typically does not cause computational problems. However, if issues occur when using the full dataset, we could attempt to minimize transformations by reducing the amount of data that needs to be shuffled.